In [21]:
### Pyhton 3.7.1 ###
#-----L2 ANOVA------
#~~packages
# overall stats
import scipy.stats as stats
# dataframe
import pandas as pd
# array/math
import numpy as np
#time
import time
tStart = time.time()
#using scipy.statsmodels for modeling
import statsmodels.api as sm
from statsmodels.formula.api import ols
# post hoc
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
# plot
import matplotlib.pyplot as plt
#data visualization
import seaborn as sns

In [22]:
#~~ loading data
df_raw = pd.read_csv(r"C:\Users\johnson_yang\Desktop\WOW_data_forStudent.csv")#改成自己的檔案位置
print(df_raw.head())

   Unnamed: 0       ID  ethnic  gender  senson  school_3  school_2  school_1  \
0           3  1041002  Human        0       1       206       204       102   
1          15  1041007  Human        0       1       101       101       101   
2          16  1041008  Human        0       1       103       103       103   
3          30  1041012  Human        1       1       104       104       104   
4          64  1041020  Human        0       1       107       107       107   

   eco_3  eco_2   ...    item12  item13  item14  item15  item16  item17  \
0      0      0   ...         4       1       4       4       1       5   
1      1      1   ...         5       1       5       5       1       5   
2      0      0   ...         4       1       4       4       1       5   
3      1      1   ...         3       1       2       2       1       2   
4      0      0   ...         5       1       5       5       1       5   

   item19  item20  item21  item22  
0       4       4       1       

In [23]:
df = df_raw[['ID','ethnic','reading_3']]
print('---------------------Data info_head---------------------')
print(df.head())
print('---------------------Data type---------------------')
print(df.dtypes)
print('---------------------ID info---------------------')
print(df['ID'].describe())
print('---------------------convert to string---------------------')
df['ID']=df['ID'].astype(str)
print(df.dtypes)
print('---------------------ethnic info---------------------')
print(df['ethnic'].describe())
print('---------------------ethnic counts---------------------')
print(df['ethnic'].value_counts())
#print('---------------------reading_3 info---------------------')
#print(rp.summary_cont(df['reading_3']))
#print('---------------------------------------------------------')

---------------------Data info_head---------------------
        ID  ethnic  reading_3
0  1041002  Human         492
1  1041007  Human         494
2  1041008  Human         523
3  1041012  Human         476
4  1041020  Human         515
---------------------Data type---------------------
ID            int64
ethnic       object
reading_3     int64
dtype: object
---------------------ID info---------------------
count    1.930000e+02
mean     1.041097e+06
std      5.585845e+01
min      1.041001e+06
25%      1.041049e+06
50%      1.041097e+06
75%      1.041145e+06
max      1.041193e+06
Name: ID, dtype: float64
---------------------convert to string---------------------
ID           object
ethnic       object
reading_3     int64
dtype: object
---------------------ethnic info---------------------
count        193
unique         3
top       Human 
freq          87
Name: ethnic, dtype: object
---------------------ethnic counts---------------------
Human      87
Undead     64
Orc        42
Name

C:\Users\johnson_yang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [4]:
#~~ normality, homogeneity
# python 沒有R的psych套件可以直接把類別變數做頻率計算，因此要轉換data type

#freq_count
df['ethnic']=df['ethnic'].str.replace('Human','1')
df['ethnic']=df['ethnic'].str.replace('Orc','2')
df['ethnic']=df['ethnic'].str.replace('Undead','3')
#convert data type
df['ethnic']=df['ethnic'].astype(int)

C:\Users\johnson_yang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\johnson_yang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\johnson_yang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [5]:
#descriptive stats (normality)
print(stats.describe(df['ethnic']))
print(stats.describe(df['reading_3']))
print('---------------------------------------------------------')

DescribeResult(nobs=193, minmax=(1, 3), mean=1.8808290155440415, variance=0.7721826424870466, skewness=0.23341889971723911, kurtosis=-1.658474100053491)
DescribeResult(nobs=193, minmax=(423, 543), mean=499.3264248704663, variance=364.53351683937825, skewness=-0.9125744541449188, kurtosis=1.6503736513748368)
---------------------------------------------------------


In [6]:
#homogeneity
# python要先把data做分組才能一組一組看變異數同質性(這裡暫時assign成df2做同質性檢定)
#group data
df2=df[['ethnic','reading_3']]
df2=df2.groupby(['ethnic'])

df2_human = df2.get_group(1)
df2_orc = df2.get_group(2)
df2_undead = df2.get_group(3)
        
print(stats.bartlett(df2_human['reading_3'],df2_orc['reading_3'], df2_undead['reading_3']))
print(stats.levene(df2_human['reading_3'],df2_orc['reading_3'],df2_undead['reading_3']))

BartlettResult(statistic=3.39757749980998, pvalue=0.1829049335526695)
LeveneResult(statistic=0.579736693646694, pvalue=0.5610333441234336)


In [7]:
#~~ one way ANOVA
results = ols('reading_3 ~ C(ethnic)', data=df).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              reading_3   R-squared:                       0.154
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                     17.25
Date:                Thu, 21 Mar 2019   Prob (F-statistic):           1.30e-07
Time:                        17:05:43   Log-Likelihood:                -826.47
No. Observations:                 193   AIC:                             1659.
Df Residuals:                     190   BIC:                             1669.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        505.5287      1.893    267.

In [19]:
# ANOVA table
aov_table= sm.stats.anova_lm(results, typ=1)
print(aov_table)

              df        sum_sq      mean_sq          F        PR(>F)
C(ethnic)    2.0  10757.617935  5378.808968  17.253505  1.302420e-07
Residual   190.0  59232.817298   311.751670        NaN           NaN


## plot
sns.boxplot(x='ethnic',y='reading_3', data=df,hue='ethnic')
sns.catplot(x="ethnic",y="reading_3",hue="ethnic", capsize=.2, height=6, aspect=.75,kind="point",data=df)
plt.show()

In [10]:
#~~ post-hoc
mc = MultiComparison(df['reading_3'], df['ethnic'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower    upper   reject
-----------------------------------------------
  1      2    -19.4811 -27.3184 -11.6438  True 
  1      3    -5.9194  -12.7884  0.9497  False 
  2      3    13.5618   5.2786  21.8449   True 
-----------------------------------------------


In [11]:
# p values
from statsmodels.stats.libqsturng import psturng
print('---------------------P-value---------------------')
print(psturng(np.abs(mc_results.meandiffs / mc_results.std_pairs), len(mc_results.groupsunique), mc_results.df_total))

---------------------P-value---------------------
[0.001      0.10655048 0.001     ]


In [12]:
#~~ eta squared
#@jit
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    cols = ['sum_sq', 'df', 'mean_sq', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

print(anova_table(aov_table))
print('----------------------------------------')

                 sum_sq     df      mean_sq          F        PR(>F)  \
C(ethnic)  10757.617935    2.0  5378.808968  17.253505  1.302420e-07   
Residual   59232.817298  190.0   311.751670        NaN           NaN   

             eta_sq  omega_sq  
C(ethnic)  0.153701  0.144151  
Residual        NaN       NaN  
----------------------------------------


In [13]:
tEnd = time.time()
print("It cost",tEnd-tStart,"sec" )

It cost 2.076333999633789 sec
